In [ ]:
import logging
import os

# Import our modules
from calibration import calibrate_vix_model
from pricing import (
    GenericMCPricer,
    VIXDedicatedMCPricer,
    load_vix_data, 
    infer_current_regime, 
    create_regime_process
)

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

data_path = os.path.join('..', 'data', 'vix_data.csv')
vix_df = load_vix_data(data_path)
vix_values = vix_df['VIX'].values

logger.info("Calibrating VIX model...")
calibrated_params, log_likelihoods = calibrate_vix_model(vix_values)
# calibrated_params = initialize_vix_params(vix_values, num_regimes=2)

logger.info("Inferring current regime...")
current_regime, regime_probs = infer_current_regime(vix_values, calibrated_params)
logger.info(f"Most likely current regime: {current_regime}")
logger.info(f"Regime probabilities: {regime_probs}")

logger.info("Creating regime process...")
regime_process = create_regime_process(calibrated_params)

current_vix = vix_values[-1]
strike_price = 13.0  # ATM option
expiry = 30  # 30 days to expiration

logger.info("Done")

In [ ]:
logger.info("Creating VIX generic MC pricer...")
pricer = GenericMCPricer(regime_process)
maturity = expiry / 252.0

logger.info("Pricing VIX options...")
for strike in [19, 21, 23]:
    call_price = pricer.price_european(
        initial_val=current_vix / 100.0,
        initial_regime=current_regime,
        strike=strike,
        maturity=maturity,
        option_type='call'
    )
    put_price = pricer.price_european(
        initial_val=current_vix / 100.0,
        initial_regime=current_regime,
        strike=strike,
        maturity=maturity,
        option_type='put'
    )
    
    logger.info(f" Strike {strike:.1f}, Expiry {expiry} days:")
    logger.info(f"  Call: {call_price:.2f}")
    logger.info(f"  Put:  {put_price:.2f}")


In [ ]:
logger.info("Creating VIX dedicated MC pricer...")
pricer = VIXDedicatedMCPricer(regime_process)

logger.info("Pricing VIX options...")
for strike in [19, 21, 23]:
    call_price = pricer.price_vix_option(
        initial_vix=current_vix,
        initial_regime=current_regime,
        strike=strike,
        expiry_days=expiry,
        option_type='call'
    )
    put_price = pricer.price_vix_option(
        initial_vix=current_vix,
        initial_regime=current_regime,
        strike=strike,
        expiry_days=expiry,
        option_type='put'
    )
    
    logger.info(f"Strike {strike:.1f}, Expiry {expiry} days:")
    logger.info(f"  Call: {call_price:.2f}")
    logger.info(f"  Put:  {put_price:.2f}")